In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from erisk.utils import get_spark

spark = get_spark()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/31 14:56:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/31 14:56:02 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [3]:
bucket = "gs://dsgt-clef-erisk-2024"
test_df = spark.read.parquet(f"{bucket}/task1/parquet/test")
test_df.show()

+--------+--------------------+--------------------+--------------------+---------------+--------+
|   DOCNO|                POST|                 PRE|                TEXT|_corrupt_record|filename|
+--------+--------------------+--------------------+--------------------+---------------+--------+
|   0_0_6|I'm trying to wor...|\n\nAlthough most...|I will not tell m...|           NULL|s_0.trec|
|456_1_12|You're not like '...|In general though...|Oh, and if you're...|           NULL|s_1.trec|
| 764_1_5|Maybe it's one of...|My past experienc...|But I still want ...|           NULL|s_1.trec|
|651_0_28|\n\nWe all run ba...|\n\nSo this woman...|I couldn't even i...|           NULL|s_1.trec|
| 268_1_3| Both were great,...|\n\nI've only had...|One a couple year...|           NULL|s_1.trec|
|364_0_12|I started opening...|Even though I too...|Which I can under...|           NULL|s_1.trec|
|765_0_33|Nowhere on my inv...|I ask about the t...|Words can not exp...|           NULL|s_1.trec|
|409_0_18|

In [4]:
test_df.count()

15542200

In [5]:
# lets also load up the train df
train_df = spark.read.parquet(f"{bucket}/task1/parquet/train")
train_df.show()
train_df.count()

+--------+--------------------+--------+
|   DOCNO|                TEXT|filename|
+--------+--------------------+--------+
| s_0_0_0|    1.ye katiliyorum|s_0.trec|
| s_0_1_0|ok haklsn abi gol...|s_0.trec|
| s_0_2_0| almanca yarrak gibi|s_0.trec|
| s_0_3_0|hani u oyunlarn e...|s_0.trec|
| s_0_3_1|dead cellste ygda...|s_0.trec|
| s_0_3_2|bunlarn bir dili ...|s_0.trec|
| s_0_4_0|lnce diriltiyor s...|s_0.trec|
| s_0_6_0|       ziya gzel sal|s_0.trec|
| s_0_7_0|  artk dedem deilsin|s_0.trec|
| s_0_8_0|sorma bizim matem...|s_0.trec|
| s_0_9_0|240 Volt FUCKMAST...|s_0.trec|
|s_0_10_0|bunlar nerden evi...|s_0.trec|
|s_0_11_0|beynine gidecek k...|s_0.trec|
|s_0_12_0|semeyen vizyonsuz...|s_0.trec|
|s_0_13_0|       ok haklsn abi|s_0.trec|
|s_0_14_0|ilkokul zamanlari...|s_0.trec|
|s_0_15_0|iliki kurmakta zo...|s_0.trec|
|s_0_15_1|liseye giden bir ...|s_0.trec|
|s_0_15_2|tipimin ve kiilii...|s_0.trec|
|s_0_15_3|ben insanlarla ko...|s_0.trec|
+--------+--------------------+--------+
only showing top

4264693

In [9]:
from pyspark.sql import functions as F

total_df = (
    train_df.select(
        "DOCNO",
        "TEXT",
        "filename",
        F.lit("train").alias("dataset"),
    )
    .union(
        test_df.select(
            "DOCNO",
            F.concat(
                F.coalesce(F.col("PRE"), F.lit("")),
                F.coalesce(F.col("TEXT"), F.lit("")),
                F.coalesce(F.col("POST"), F.lit("")),
            ).alias("TEXT"),
            "filename",
            F.lit("test").alias("dataset"),
        ),
    )
    .where("filename is not null")
    .where("TEXT is not null")
)

subset_df = total_df.limit(100).cache()
subset_df.show()

+--------+--------------------+--------+-------+
|   DOCNO|                TEXT|filename|dataset|
+--------+--------------------+--------+-------+
| s_0_0_0|    1.ye katiliyorum|s_0.trec|  train|
| s_0_1_0|ok haklsn abi gol...|s_0.trec|  train|
| s_0_2_0| almanca yarrak gibi|s_0.trec|  train|
| s_0_3_0|hani u oyunlarn e...|s_0.trec|  train|
| s_0_3_1|dead cellste ygda...|s_0.trec|  train|
| s_0_3_2|bunlarn bir dili ...|s_0.trec|  train|
| s_0_4_0|lnce diriltiyor s...|s_0.trec|  train|
| s_0_6_0|       ziya gzel sal|s_0.trec|  train|
| s_0_7_0|  artk dedem deilsin|s_0.trec|  train|
| s_0_8_0|sorma bizim matem...|s_0.trec|  train|
| s_0_9_0|240 Volt FUCKMAST...|s_0.trec|  train|
|s_0_10_0|bunlar nerden evi...|s_0.trec|  train|
|s_0_11_0|beynine gidecek k...|s_0.trec|  train|
|s_0_12_0|semeyen vizyonsuz...|s_0.trec|  train|
|s_0_13_0|       ok haklsn abi|s_0.trec|  train|
|s_0_14_0|ilkokul zamanlari...|s_0.trec|  train|
|s_0_15_0|iliki kurmakta zo...|s_0.trec|  train|
|s_0_15_1|liseye gid

24/03/31 15:07:32 WARN CacheManager: Asked to cache already cached data.


In [12]:
from pyspark import keyword_only
from pyspark.ml import Transformer
from pyspark.ml.param.shared import (
    HasInputCol,
    HasOutputCol,
    Param,
    Params,
    TypeConverters,
)
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
from pyspark.sql.types import ArrayType, FloatType
import numpy as np
from pyspark.ml.functions import predict_batch_udf

"""
Wrapper for BERT to add it to the pipeline
"""


class BertTransformer(
    Transformer, HasInputCol, HasOutputCol, DefaultParamsReadable, DefaultParamsWritable
):
    input_col = Param(
        Params._dummy(),
        "input_col",
        "input column name.",
        typeConverter=TypeConverters.toString,
    )
    output_col = Param(Params._dummy(), "output_col", "output column name.")

    def make_predict_fn(self):
        """Return PredictBatchFunction"""
        from sentence_transformers import SentenceTransformer

        model = SentenceTransformer("all-MiniLM-L6-v2")

        def predict(inputs: np.ndarray) -> np.ndarray:
            return model.encode(inputs)

        return predict

    @keyword_only
    def __init__(self, input_col: str = "input", output_col: str = "output"):
        super().__init__()
        self._setDefault(inputCol=input_col, outputCol=output_col)

    def _transform(self, df: DataFrame):

        input_col = self.getInputCol()
        output_col = self.getOutputCol()

        # batch prediction UDF
        apply_predict_batch = predict_batch_udf(
            make_predict_fn=self.make_predict_fn,
            return_type=ArrayType(FloatType()),
            batch_size=8,
        )

        return df.withColumn(output_col, apply_predict_batch(input_col))

In [14]:
from pyspark.ml import Pipeline

transformer = BertTransformer(input_col="TEXT", output_col="embeddings")
pipeline = Pipeline(stages=[transformer])

pipeline_model = pipeline.fit(subset_df)
%time pipeline_model.transform(subset_df).show()

+--------+--------------------+--------+-------+--------------------+
|   DOCNO|                TEXT|filename|dataset|          embeddings|
+--------+--------------------+--------+-------+--------------------+
| s_0_0_0|    1.ye katiliyorum|s_0.trec|  train|[0.0180698, 0.004...|
| s_0_1_0|ok haklsn abi gol...|s_0.trec|  train|[-0.13028191, 0.0...|
| s_0_2_0| almanca yarrak gibi|s_0.trec|  train|[-0.020124434, 0....|
| s_0_3_0|hani u oyunlarn e...|s_0.trec|  train|[-0.046469197, 0....|
| s_0_3_1|dead cellste ygda...|s_0.trec|  train|[-0.07952062, 0.0...|
| s_0_3_2|bunlarn bir dili ...|s_0.trec|  train|[0.0138995685, 0....|
| s_0_4_0|lnce diriltiyor s...|s_0.trec|  train|[-0.05409137, 0.0...|
| s_0_6_0|       ziya gzel sal|s_0.trec|  train|[-0.11549768, 0.1...|
| s_0_7_0|  artk dedem deilsin|s_0.trec|  train|[-0.11340375, 0.1...|
| s_0_8_0|sorma bizim matem...|s_0.trec|  train|[-0.024256464, 0....|
| s_0_9_0|240 Volt FUCKMAST...|s_0.trec|  train|[-0.06978254, 0.0...|
|s_0_10_0|bunlar ner